In [40]:
%load_ext autoreload
%autoreload 2

import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
from sklearn.preprocessing import MinMaxScaler
import xarray as xr
#import pymaise 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
from pyMAISE.datasets import load_anomaly
from pyMAISE.preprocessing import train_test_split, scale_data, one_hot_encode, SplitSequence
import pyMAISE as mai

In [42]:
global_settings = mai.init(
    problem_type=mai.ProblemType.CLASSIFICATION,   # Define a regression problem
    cuda_visible_devices="-1"                  # Use CPU only/ Delete line when run on GPU
)

In [48]:
# call handler
main_path = "../../../anomaly_data/"
input_path = f"{main_path}DTL.npy"
output_path = f"{main_path}DTL_labels.npy"
#print(mai.__file__)
#False, False, False: WORKS       X.shape = (1077, 4500, 14), Y.shape = (1077, 1)  one hot
#False, True, False: WORKS       X.shape = (1077, 4500, 14), Y.shape = (1077, 1)
#False, False, True:  ?      X.shape = (1077, 4500, 14), Y.shape = (1077, 4500, 1)
#False, True, True:   ?      X.shape = (1077, 4500, 14), Y.shape = (1077, 4500, 1)
#True, False, True: WORKS        X.shape = (4846500, 14), Y.shape = (4846500, 1)
#True, True, True: WORKS         X.shape = (4846500, 14), Y.shape = (4846500, 1)
cond1 = True
cond2 = False
cond3 = True
inputs, outputs = load_anomaly([input_path, output_path], cond1, cond2, cond3)


In [49]:
inputs

<xarray.DataArray (timesteps: 4846500, features: 14)>
array([[-6.07100e+00,  5.55120e+00, -1.42220e+00, ..., -4.60370e-02,
         2.17170e+03,  1.27100e+00],
       [-4.85030e+00,  4.33040e+00, -2.64290e+00, ...,  1.50000e-02,
         2.17250e+03, -1.33776e+01],
       [-4.85030e+00,  1.88890e+00, -1.42220e+00, ...,  4.55190e-02,
         2.17320e+03, -8.49500e+00],
       ...,
       [-2.17470e+00, -7.57620e+00, -3.39550e+00, ..., -1.07070e-01,
         2.17220e+03,  1.19730e+01],
       [-4.61620e+00, -3.91400e+00, -5.83700e+00, ..., -1.68110e-01,
         2.17180e+03,  9.53200e+00],
       [-3.39550e+00, -6.35550e+00, -5.83700e+00, ..., -4.60370e-02,
         2.17150e+03,  9.53200e+00]], dtype=float32)
Coordinates:
  * timesteps  (timesteps) int64 0 1 2 3 4 ... 4846496 4846497 4846498 4846499
  * features   (features) <U18 'A+IGBT-I: current' ... 'DV/DT'

In [50]:
outputs

<xarray.DataArray (timesteps: 4846500, features: 1)>
array([['Run'],
       ['Run'],
       ['Run'],
       ...,
       ['Fault'],
       ['Fault'],
       ['Fault']], dtype=object)
Coordinates:
  * timesteps  (timesteps) int64 0 1 2 3 4 ... 4846496 4846497 4846498 4846499
  * features   (features) <U5 'Class'

In [51]:
outputs = one_hot_encode(outputs)
outputs

<xarray.DataArray (timesteps: 4846500, features: 2)>
array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])
Coordinates:
  * timesteps  (timesteps) int64 0 1 2 3 4 ... 4846496 4846497 4846498 4846499
  * features   (features) object 'Class_Fault' 'Class_Run'

In [53]:
data = xr.concat([inputs, outputs], dim="features")
data

<xarray.DataArray (timesteps: 4846500, features: 16)>
array([[ -6.0710001 ,   5.55119991,  -1.42219996, ...,   1.27100003,
          0.        ,   1.        ],
       [ -4.85029984,   4.33039999,  -2.64289999, ..., -13.37759972,
          0.        ,   1.        ],
       [ -4.85029984,   1.88890004,  -1.42219996, ...,  -8.49499989,
          0.        ,   1.        ],
       ...,
       [ -2.17470002,  -7.57620001,  -3.39549994, ...,  11.97299957,
          1.        ,   0.        ],
       [ -4.61619997,  -3.91400003,  -5.83699989, ...,   9.53199959,
          1.        ,   0.        ],
       [ -3.39549994,  -6.35550022,  -5.83699989, ...,   9.53199959,
          1.        ,   0.        ]])
Coordinates:
  * timesteps  (timesteps) int64 0 1 2 3 4 ... 4846496 4846497 4846498 4846499
  * features   (features) object 'A+IGBT-I: current' ... 'Class_Run'

In [58]:
split_sequence = SplitSequence(
    10, 
    1, 
    0, 
    sequence_inputs=data.coords["features"].values[:-2], 
    sequence_outputs=data.coords["features"].values[-2:],
)
inputs, outputs = split_sequence.split(data)

inputs

<xarray.DataArray (samples: 4846491, timesteps: 10, features: 14)>
array([[[-6.07100010e+00,  5.55119991e+00, -1.42219996e+00, ...,
         -4.60369997e-02,  2.17169995e+03,  1.27100003e+00],
        [-4.85029984e+00,  4.33039999e+00, -2.64289999e+00, ...,
          1.49999997e-02,  2.17250000e+03, -1.33775997e+01],
        [-4.85029984e+00,  1.88890004e+00, -1.42219996e+00, ...,
          4.55190018e-02,  2.17319995e+03, -8.49499989e+00],
        ...,
        [-3.62960005e+00,  1.88890004e+00, -5.08440018e+00, ...,
         -4.60369997e-02,  2.17239990e+03, -1.17100000e+00],
        [-7.29180002e+00,  1.88890004e+00, -3.86360002e+00, ...,
          1.49999997e-02,  2.17230005e+03, -1.33781004e+01],
        [-7.29180002e+00,  3.10969996e+00, -2.64289999e+00, ...,
         -4.60369997e-02,  2.17280005e+03, -1.17100000e+00]],

       [[-4.85029984e+00,  4.33039999e+00, -2.64289999e+00, ...,
          1.49999997e-02,  2.17250000e+03, -1.33775997e+01],
        [-4.85029984e+00,  1.88890004e+00, -1.42219996e+00, ...,
          4.55190018e-02,  2.17319995e+03, -8.49499989e+00],
        [-7.29180002e+00,  1.88890004e+00, -1.42219996e+00, ...,
          4.55190018e-02,  2.17300000e+03, -2.31439991e+01],
...
        [-4.61619997e+00, -8.79699993e+00, -4.61619997e+00, ...,
          1.49999997e-02,  2.17180005e+03,  4.64900017e+00],
        [-2.17470002e+00, -7.57620001e+00, -3.39549994e+00, ...,
         -1.07069999e-01,  2.17219995e+03,  1.19729996e+01],
        [-4.61619997e+00, -3.91400003e+00, -5.83699989e+00, ...,
         -1.68109998e-01,  2.17180005e+03,  9.53199959e+00]],

       [[-7.05770016e+00, -7.57620001e+00, -3.39549994e+00, ...,
          1.06559999e-01,  2.17180005e+03,  7.09000015e+00],
        [-4.61619997e+00, -6.35550022e+00, -7.05770016e+00, ...,
         -4.60369997e-02,  2.17160010e+03,  7.09100008e+00],
        [-3.39549994e+00, -8.79699993e+00, -2.17470002e+00, ...,
         -4.60369997e-02,  2.17180005e+03,  1.19740000e+01],
        ...,
        [-2.17470002e+00, -7.57620001e+00, -3.39549994e+00, ...,
         -1.07069999e-01,  2.17219995e+03,  1.19729996e+01],
        [-4.61619997e+00, -3.91400003e+00, -5.83699989e+00, ...,
         -1.68109998e-01,  2.17180005e+03,  9.53199959e+00],
        [-3.39549994e+00, -6.35550022e+00, -5.83699989e+00, ...,
         -4.60369997e-02,  2.17150000e+03,  9.53199959e+00]]])
Coordinates:
  * samples    (samples) int64 0 1 2 3 4 ... 4846487 4846488 4846489 4846490
  * timesteps  (timesteps) int64 0 1 2 3 4 5 6 7 8 9
  * features   (features) object 'A+IGBT-I: current' ... 'DV/DT'

In [34]:
inputs = np.array(inputs) #add names for 14 from paper

#inputs


In [35]:
if (cond1 == False and cond2 == False and cond3 == True) or (cond1 == False and cond2 == True and cond3 == True):  

# Assuming inputs is a 3D numpy array with shape (number, timesteps, columns)
    number, timesteps, columns = inputs.shape

# Column names should be the same length as the size of the third dimension of inputs.
    column_names_in = [
        'A+IGBT-I: current', 'A+*IGBT-I: current', 'B+IGBT-I: current',
        'B+*IGBT-I: current', 'C+IGBT-I: current', 'C+*IGBT-I: current',
        'A-Flux', 'B-Flux', 'C-Flux', 'Mod-V', 'Mod-I', 'CB-I', 'CB-V', 'DV/DT'
    ]

# Ensure the list of column names matches the size of the third dimension:
    assert len(column_names_in) == columns

# Define coordinates for the DataArray
    coords = {
        "number": np.arange(number),  # Number coordinate (size matches first dim of inputs)
        "timesteps": np.arange(timesteps),  # Timesteps coordinate (second dim)
        "innn": column_names_in  # Column names (third dim)
    }

# Define dimensions
    dims = ("number", "timesteps", "innn")

# Create the DataArray
    inp = xr.DataArray(
        inputs,
        coords=coords,
        dims=dims
    )
   
    coords = {
        "number": np.arange(number),  # Number coordinate (size matches first dim of inputs)
        "timesteps": np.arange(timesteps),  # Timesteps coordinate (second dim)
        "innn": "???"  # Column names (third dim)
    }

# Define dimensions
    dims = ("number", "timesteps", "Faulty")

# Create the DataArray
    out = xr.DataArray(
        outputs,
        coords=coords,
        dims=dims
    )
        
# Define coordinates for the DataArray
    coords = {
        "number": np.arange(number),  # Number coordinate (size matches first dim of inputs)
        "timesteps": np.arange(timesteps),  # Timesteps coordinate (second dim)
        "Faulty": "Fault Type"  # Column names (third dim)
    }

# Define dimensions
    dims = ("number", "timesteps", "Faulty")

# Create the DataArray
    out = xr.DataArray(
        outputs,
        coords=coords,
        dims=dims
    )
    out
    
else:
    print("error")


error


In [36]:
if (cond1 == False and cond2 == False and cond3 == False) or (cond1 == False and cond2 == True and cond3 == False):  

# Assuming inputs is a 3D numpy array with shape (number, timesteps, columns)
    number, timesteps, columns = inputs.shape

# Column names should be the same length as the size of the third dimension of inputs.
    column_names_in = [
        'A+IGBT-I: current', 'A+*IGBT-I: current', 'B+IGBT-I: current',
        'B+*IGBT-I: current', 'C+IGBT-I: current', 'C+*IGBT-I: current',
        'A-Flux', 'B-Flux', 'C-Flux', 'Mod-V', 'Mod-I', 'CB-I', 'CB-V', 'DV/DT'
    ]

# Ensure the list of column names matches the size of the third dimension:
    assert len(column_names_in) == columns

# Define coordinates for the DataArray
    coords = {
        "number": np.arange(number),  # Number coordinate (size matches first dim of inputs)
        "timesteps": np.arange(timesteps),  # Timesteps coordinate (second dim)
        "col": column_names_in  # Column names (third dim)
    }

# Define dimensions
    dims = ("number", "timesteps", "col")

# Create the DataArray
    inp = xr.DataArray(
        inputs,
        coords=coords,
        dims=dims
    )
   
    
    column_names_out = ['Fault Status']
    out = xr.DataArray(
        outputs,
        coords={"lon": np.arange(inputs.shape[0]), "col": column_names_out},
        dims=("lon", "col"),
    )
    #out.col.attrs["name"] = "columns"
    out
    
    
    
else:
    print("error")

In [37]:
if (cond1 == True and cond2 == True and cond3 == True) or (cond1 == True and cond2 == False and cond3 == True):  
    column_names_in = ['A+IGBT-I: current', 
                'A+*IGBT-I: current', 
                'B+IGBT-I: current', 
                'B+*IGBT-I: current', 
                'C+IGBT-I: current', 
                'C+*IGBT-I: current', 
                'A-Flux', 
                'B-Flux', 
                'C-Flux', 
                'Mod-V', 
                'Mod-I', 
                'CB-I', 
                'CB-V', 
                'DV/DT']

    inp = xr.DataArray(
            inputs,
            coords={"lon": np.arange(inputs.shape[0]), "col": column_names_in},
            dims=("lon", "col"),
    )
    inp.col.attrs["Timestepxnumber"] = "columns"
    inputs.shape[0]
    inp


    column_names_out = ['Fault Status']
    out = xr.DataArray(
        outputs,
        coords={"lon": np.arange(inputs.shape[0]), "col": column_names_out},
        dims=("lon", "col"),
    )
    #out.col.attrs["name"] = "columns"

    out

    print("hi")
else:
    print("error")

error


In [38]:
inp

<xarray.DataArray (number: 1077, timesteps: 4500, col: 14)>
array([[[-6.07100e+00,  5.55120e+00, -1.42220e+00, ..., -4.60370e-02,
          2.17170e+03,  1.27100e+00],
        [-4.85030e+00,  4.33040e+00, -2.64290e+00, ...,  1.50000e-02,
          2.17250e+03, -1.33776e+01],
        [-4.85030e+00,  1.88890e+00, -1.42220e+00, ...,  4.55190e-02,
          2.17320e+03, -8.49500e+00],
        ...,
        [ 4.91560e+00,  7.99260e+00, -7.52590e+00, ..., -4.60370e-02,
          2.09030e+03,  2.81300e+01],
        [ 1.25340e+00,  1.04340e+01, -8.74660e+00, ..., -4.60370e-02,
          2.09320e+03,  3.30100e+01],
        [ 3.69490e+00,  5.55120e+00, -7.52590e+00, ..., -4.60370e-02,
          2.09200e+03,  2.81300e+01]],

       [[-6.07100e+00,  6.68200e-01, -6.30510e+00, ...,  4.55190e-02,
          2.13900e+03, -1.33778e+01],
        [-6.07100e+00,  1.88890e+00, -5.08440e+00, ...,  4.55190e-02,
          2.13920e+03, -1.82608e+01],
        [-9.73330e+00, -5.52540e-01, -6.30510e+00, ...,  1.50000e-02,
          2.13850e+03, -1.33778e+01],
...
        [-2.17470e+00, -6.35550e+00, -5.83700e+00, ...,  1.50000e-02,
          2.17330e+03,  9.53200e+00],
        [-2.17470e+00, -3.91400e+00, -5.83700e+00, ...,  1.50000e-02,
          2.17350e+03,  9.53200e+00],
        [-9.54010e-01, -7.57620e+00, -3.39550e+00, ...,  1.50000e-02,
          2.17250e+03,  1.68560e+01]],

       [[-4.61620e+00, -1.24590e+01, -8.27850e+00, ..., -1.07070e-01,
          2.16820e+03,  1.68560e+01],
        [-2.17470e+00, -5.13470e+00, -4.61620e+00, ...,  1.50000e-02,
          2.16770e+03,  1.44150e+01],
        [-7.05770e+00, -8.79700e+00, -4.61620e+00, ...,  4.55190e-02,
          2.16770e+03,  9.53200e+00],
        ...,
        [-2.17470e+00, -7.57620e+00, -3.39550e+00, ..., -1.07070e-01,
          2.17220e+03,  1.19730e+01],
        [-4.61620e+00, -3.91400e+00, -5.83700e+00, ..., -1.68110e-01,
          2.17180e+03,  9.53200e+00],
        [-3.39550e+00, -6.35550e+00, -5.83700e+00, ..., -4.60370e-02,
          2.17150e+03,  9.53200e+00]]], dtype=float32)
Coordinates:
  * number     (number) int64 0 1 2 3 4 5 6 ... 1071 1072 1073 1074 1075 1076
  * timesteps  (timesteps) int64 0 1 2 3 4 5 6 ... 4494 4495 4496 4497 4498 4499
  * col        (col) <U18 'A+IGBT-I: current' 'A+*IGBT-I: current' ... 'DV/DT'

In [39]:
# Assuming out is your xarray DataArray
out

<xarray.DataArray (lon: 1077, col: 1)>
array([['Run'],
       ['Run'],
       ['Run'],
       ...,
       ['Fault'],
       ['Fault'],
       ['Fault']], dtype=object)
Coordinates:
  * lon      (lon) int64 0 1 2 3 4 5 6 7 ... 1070 1071 1072 1073 1074 1075 1076
  * col      (col) <U12 'Fault Status'

In [284]:
if out.ndim == 2:
    print(out.ndim)
    out_onehot = one_hot_encode(out)
    
else:
    print("fix")

2


In [292]:
out_onehot

<xarray.DataArray (lon: 1077, col: 2)>
array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])
Coordinates:
  * lon      (lon) int64 0 1 2 3 4 5 6 7 ... 1070 1071 1072 1073 1074 1075 1076
  * col      (col) object 'Fault Status_Fault' 'Fault Status_Run'

In [286]:
#TESTING SPLIT SEQUENCE
from pyMAISE import datasets, preprocessing
#_, perturbed = datasets.load_loca(stack_series=True)
#perturbed.shape
#perturbed


In [287]:
#perturbed.shape

In [288]:
inp.shape

(1077, 4500, 14)

In [289]:
sequence_outputs = [
                'A+IGBT-I: current', 
                'A+*IGBT-I: current', 
                'B+IGBT-I: current', 
                'B+*IGBT-I: current', 
                'C+IGBT-I: current', 
                'C+*IGBT-I: current', 
                'A-Flux', 
                'B-Flux', 
                'C-Flux', 
                'Mod-V', 
                'Mod-I', 
                'CB-I', 
                'CB-V', 
                'DV/DT'
]
split_sequences = preprocessing.SplitSequence(
        5,
        1,
        1,
        sequence_outputs=sequence_outputs
)
perturbed_input, perturbed_output = split_sequences.split(inp)
perturbed_input.shape



(1077, 4495, 70)

In [290]:
perturbed_output.shape

(1077, 4495, 14)

In [291]:


# one hot 
#split sequence 
 # scale data the inputs 
    
    # all of preprocessing
    
    
#initialize ML methods 
#
